In [335]:
import pandas as pd
import csv
import json
import elasticsearch

In [336]:
#confirm connection to elasticsearch (running in docker)
es = Elasticsearch(
    hosts=[
            "https://localhost:9200"
    ],
    basic_auth=("elastic", "Cu58a5GaVEavNXPpPAZS"),
    
    verify_certs=True,
    ca_certs="./http_ca.crt",
)
es.ping()

True

In [208]:
es.info()

ObjectApiResponse({'name': '4bfe196979f0', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'jNku1f3eRm2S4WMcC1TB7g', 'version': {'number': '8.11.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6f9ff581fbcde658e6f69d6ce03050f060d1fd0c', 'build_date': '2023-11-11T10:05:59.421038163Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [274]:
#get data
trainPath = '/Users/jamester/Documents/DataAnalysis/ELK/LoanTrain.csv'
testPath = '/Users/jamester/Documents/DataAnalysis/ELK/LoanTest.csv'

train = pd.read_csv(trainPath); print(train.shape)
test = pd.read_csv(testPath); print(test.shape)

(614, 13)
(367, 12)


In [275]:
train.head(
)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [276]:
train.shape

(614, 13)

In [277]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [325]:
mappings = {'properties': {
            'Loan_ID': {'type':'keyword'},
            'Gender': {'type':'keyword'},
            'Married': {'type':'keyword'},
            'Dependents':{'type':'keyword'},
            'Education': {'type':'keyword'},
            'Self_Employed':{'type':'keyword'},
            'ApplicantIncome': {'type':'float'},
            'CoapplicantIncome': {'type':'float'},
            'LoanAmount': {'type':'float'},
            'Loan_Amount_Term': {'type':'float'},
            'Credit_History': {'type':'float'},
            'Property_Area':{'type':'keyword'},
            'Loan_Status':{'type':'keyword'} 
        }
}

In [326]:
#create an index to load the docs to
es.indices.create(index='loan_data', mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'loan_data'})

In [327]:
#convert csv to list of dicts
csv_file = open(trainPath, 'r')
csv_reader = csv.reader(csv_file)

In [328]:
field_names = next(csv_reader)
print(field_names)

['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status']


In [329]:
data = []
for row in csv_reader:
    data.append(dict(zip(field_names, row)))

In [330]:
print(type(data))
print(data[0:5])

<class 'list'>
[{'Loan_ID': 'LP001002', 'Gender': 'Male', 'Married': 'No', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': '5849', 'CoapplicantIncome': '0', 'LoanAmount': '', 'Loan_Amount_Term': '360', 'Credit_History': '1', 'Property_Area': 'Urban', 'Loan_Status': 'Y'}, {'Loan_ID': 'LP001003', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '1', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': '4583', 'CoapplicantIncome': '1508', 'LoanAmount': '128', 'Loan_Amount_Term': '360', 'Credit_History': '1', 'Property_Area': 'Rural', 'Loan_Status': 'N'}, {'Loan_ID': 'LP001005', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'Yes', 'ApplicantIncome': '3000', 'CoapplicantIncome': '0', 'LoanAmount': '66', 'Loan_Amount_Term': '360', 'Credit_History': '1', 'Property_Area': 'Urban', 'Loan_Status': 'Y'}, {'Loan_ID': 'LP001006', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '0', 'Education':

In [331]:
csv_file.close()
#json_file.close

In [332]:
#insert train data documents into elasticsearch

for d in data:
        es.index(index='loan_data', document=d)

In [315]:
#get a single document from the new index with its created id
res = es.get(index= 'loan_data', id = 'g5HZEowBt7ngq80VFiFj')
print(res['_source'])

{'Loan_ID': 'LP001002', 'Gender': 'Male', 'Married': 'No', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': '5849', 'CoapplicantIncome': '0', 'LoanAmount': '', 'Loan_Amount_Term': '360', 'Credit_History': '1', 'Property_Area': 'Urban', 'Loan_Status': 'Y'}


In [316]:
#update document
es.update(index='loan_data', id='g5HZEowBt7ngq80VFiFj', doc={'country':'United States'})
res = es.get(index= 'loan_data', id = 'g5HZEowBt7ngq80VFiFj')
print(res['_source'])

{'Loan_ID': 'LP001002', 'Gender': 'Male', 'Married': 'No', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': '5849', 'CoapplicantIncome': '0', 'LoanAmount': '', 'Loan_Amount_Term': '360', 'Credit_History': '1', 'Property_Area': 'Urban', 'Loan_Status': 'Y', 'country': 'United States'}


In [317]:
#delete documnent
res = es.delete(index='loan_data', id='g5HZEowBt7ngq80VFiFj')
print(res)

{'_index': 'loan_data', '_id': 'g5HZEowBt7ngq80VFiFj', '_version': 3, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 615, '_primary_term': 1}


In [318]:
#search for exact match
query_results = es.search(index='loan_data', query={'match':{'Dependents':'2'}})

In [319]:
#to do: make this more readable
print(query_results)

{'took': 16, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 101, 'relation': 'eq'}, 'max_score': 1.8015635, 'hits': [{'_index': 'loan_data', '_id': 'iJHZEowBt7ngq80VFiGG', '_score': 1.8015635, '_source': {'Loan_ID': 'LP001011', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '2', 'Education': 'Graduate', 'Self_Employed': 'Yes', 'ApplicantIncome': '5417', 'CoapplicantIncome': '4196', 'LoanAmount': '267', 'Loan_Amount_Term': '360', 'Credit_History': '1', 'Property_Area': 'Urban', 'Loan_Status': 'Y'}}, {'_index': 'loan_data', '_id': 'i5HZEowBt7ngq80VFiGS', '_score': 1.8015635, '_source': {'Loan_ID': 'LP001018', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '2', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': '4006', 'CoapplicantIncome': '1526', 'LoanAmount': '168', 'Loan_Amount_Term': '360', 'Credit_History': '1', 'Property_Area': 'Urban', 'Loan_Status': 'Y'}}, {'_index': 'loan_data', '_id': 'jZ

#search for income of 5000 or more
income_search = es.search(index='loan_data', query={'regexp':{'ApplicantIncome':'[5-9]\d{3,}|[1-9]\d{4,}'}})

In [321]:
type(income_search.body)

dict

In [322]:
print(income_search.body)

{'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 195, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'loan_data', '_id': '3Xpn3osBNr2pREXOCEA5', '_score': 1.0, '_source': {'Loan_ID': 'LP001008', 'Gender': 'Male', 'Married': 'No', 'Dependents': '0', 'Education': 'Graduate', 'Self_Employed': 'No', 'ApplicantIncome': '6000', 'CoapplicantIncome': '0', 'LoanAmount': '141', 'Loan_Amount_Term': '360', 'Credit_History': '1', 'Property_Area': 'Urban', 'Loan_Status': 'Y'}}, {'_index': 'loan_data', '_id': '3npn3osBNr2pREXOCEA8', '_score': 1.0, '_source': {'Loan_ID': 'LP001011', 'Gender': 'Male', 'Married': 'Yes', 'Dependents': '2', 'Education': 'Graduate', 'Self_Employed': 'Yes', 'ApplicantIncome': '5417', 'CoapplicantIncome': '4196', 'LoanAmount': '267', 'Loan_Amount_Term': '360', 'Credit_History': '1', 'Property_Area': 'Urban', 'Loan_Status': 'Y'}}, {'_index': 'loan_data', '_id': '4npn3osBNr2pREXOCEBM', '_

In [323]:
res_hits = income_search['hits']['hits']

In [324]:
#print out returned income amounts
for num, doc in enumerate(res_hits):
    print ('\n', num, '--', doc['_source']['ApplicantIncome'])



 0 -- 6000

 1 -- 5417

 2 -- 12841

 3 -- 7660

 4 -- 5955

 5 -- 9560

 6 -- 12500

 7 -- 5649

 8 -- 5821

 9 -- 11500
